<a href="https://colab.research.google.com/github/jay-kanakia/GenAI/blob/main/08_Pytorch_project_ann_fashion_mnist_gpu_optimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d zalando-research/fashionmnist
!unzip fashionmnist.zip -d data/

Dataset URL: https://www.kaggle.com/datasets/zalando-research/fashionmnist
License(s): other
  0% 0.00/68.8M [00:00<?, ?B/s]
100% 68.8M/68.8M [00:00<00:00, 1.68GB/s]
Archive:  fashionmnist.zip
  inflating: data/fashion-mnist_test.csv  
  inflating: data/fashion-mnist_train.csv  
  inflating: data/t10k-images-idx3-ubyte  
  inflating: data/t10k-labels-idx1-ubyte  
  inflating: data/train-images-idx3-ubyte  
  inflating: data/train-labels-idx1-ubyte  


In [4]:
train_data=pd.read_csv(r'/content/data/fashion-mnist_train.csv')
test_data=pd.read_csv(r'/content/data/fashion-mnist_test.csv')

In [5]:
train_data.sample(2)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
45300,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59196,8,0,0,0,0,0,0,0,0,0,...,122,122,0,0,0,0,0,0,0,0


In [8]:
torch.manual_seed(42)

In [9]:
X_train=train_data.iloc[:,1:].values
X_test=test_data.iloc[:,1:].values

y_train=train_data.iloc[:,0].values
y_test=test_data.iloc[:,0].values

In [10]:
X_train=X_train/255
X_test=X_test/255

In [11]:
X_train_tensor=torch.from_numpy(X_train.astype(np.float32))
X_test_tensor=torch.from_numpy(X_test.astype(np.float32))

y_train_tensor=torch.from_numpy(y_train).long()
y_test_tensor=torch.from_numpy(y_test).long()

In [16]:
class MyDataset(Dataset):

  def __init__(self,features,labels):
    self.features=features
    self.labels=labels

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self,index):
    return self.features[index],self.labels[index]

In [17]:
train_dataset=MyDataset(X_train_tensor,y_train_tensor)
test_dataset=MyDataset(X_test_tensor,y_test_tensor)

In [18]:
train_loader=DataLoader(train_dataset,batch_size=256,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=256,shuffle=False)

In [29]:
class MyNN(nn.Module):
  def __init__(self,num_features):

    super().__init__()

    self.model=nn.Sequential(
        nn.Linear(num_features,128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(128,128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(128,64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(64,10)
    )

  def forward(self,X):
    return self.model(X)

In [30]:
device=torch.device('cuda')
model=MyNN(X_train_tensor.shape[1])
model=model.to(device)

epochs=200
learning_rate=0.1

loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [31]:
for epoch in range(epochs):
  total_loss=0
  for batch_features,batch_labels in train_loader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    y_pred=model(batch_features)

    loss=loss_function(y_pred,batch_labels)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    total_loss+=loss.item()

  avg_loss=total_loss/len(train_loader)
  print(f'epoch is {epoch+1} loss is {avg_loss}')

epoch is 1 loss is 0.7709842139102043
epoch is 2 loss is 0.5182094440815297
epoch is 3 loss is 0.47028000113811896
epoch is 4 loss is 0.4427720081298909
epoch is 5 loss is 0.4250760617408347
epoch is 6 loss is 0.41034996408097285
epoch is 7 loss is 0.39845730738436924
epoch is 8 loss is 0.3851976338219135
epoch is 9 loss is 0.37692542374134064
epoch is 10 loss is 0.3680352580674151
epoch is 11 loss is 0.3592206744437522
epoch is 12 loss is 0.3542935194487267
epoch is 13 loss is 0.3445947715576659
epoch is 14 loss is 0.3438805261824993
epoch is 15 loss is 0.3374674665166977
epoch is 16 loss is 0.33014744387028067
epoch is 17 loss is 0.3232270673234412
epoch is 18 loss is 0.3232814800866107
epoch is 19 loss is 0.31593550443649293
epoch is 20 loss is 0.3124228259350391
epoch is 21 loss is 0.3090076194164601
epoch is 22 loss is 0.3080366383207605
epoch is 23 loss is 0.29797991273250984
epoch is 24 loss is 0.29694672391769733
epoch is 25 loss is 0.295190410981787
epoch is 26 loss is 0.29658

In [32]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=128, bias=True)
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [33]:
total=0
correct=0
with torch.no_grad():
  for batch_features,batch_labels in test_loader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    y_pred=model(batch_features)
    y_pred=torch.argmax(y_pred,dim=1)
    total += batch_features.shape[0]
    correct += (y_pred==batch_labels).sum().item()

  print(correct/total)

0.8911


In [34]:
total=0
correct=0
with torch.no_grad():
  for batch_features,batch_labels in train_loader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    y_pred=model(batch_features)
    y_pred=torch.argmax(y_pred,dim=1)
    total += batch_features.shape[0]
    correct += (y_pred==batch_labels).sum().item()

  print(correct/total)

0.9584
